# Neural Network

In [ ]:
# import some related libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuaration
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
input_size = 784     # 28*28 = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [5]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root="../data",
                                         train=True,
                                         transform=transforms.ToTensor(),
                                         download=True)

test_dataset = torchvision.datasets.MNIST(root='../data',
                                        train=False,
                                        transform=transforms.ToTensor())

In [6]:
# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [7]:
# Fully connected meural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [8]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        # print("images.shape = {}".format(images.shape))
        # torch.Size([100, 1, 28, 28])
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}"
                 .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2529
Epoch [1/5], Step [200/600], Loss: 0.2471
Epoch [1/5], Step [300/600], Loss: 0.2069
Epoch [1/5], Step [400/600], Loss: 0.1172
Epoch [1/5], Step [500/600], Loss: 0.1980
Epoch [1/5], Step [600/600], Loss: 0.2600
Epoch [2/5], Step [100/600], Loss: 0.1533
Epoch [2/5], Step [200/600], Loss: 0.1431
Epoch [2/5], Step [300/600], Loss: 0.1007
Epoch [2/5], Step [400/600], Loss: 0.2089
Epoch [2/5], Step [500/600], Loss: 0.1153
Epoch [2/5], Step [600/600], Loss: 0.0596
Epoch [3/5], Step [100/600], Loss: 0.0498
Epoch [3/5], Step [200/600], Loss: 0.0724
Epoch [3/5], Step [300/600], Loss: 0.1529
Epoch [3/5], Step [400/600], Loss: 0.0703
Epoch [3/5], Step [500/600], Loss: 0.0826
Epoch [3/5], Step [600/600], Loss: 0.0340
Epoch [4/5], Step [100/600], Loss: 0.0907
Epoch [4/5], Step [200/600], Loss: 0.0391
Epoch [4/5], Step [300/600], Loss: 0.0608
Epoch [4/5], Step [400/600], Loss: 0.0318
Epoch [4/5], Step [500/600], Loss: 0.0671
Epoch [4/5], Step [600/600], Loss:

In [17]:
# Test the model
# In test phase, we don't need to compute gradients
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # print("_ = ", _)
        # print("predicted = ", predicted)
        # tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9
        # print("labels.size(0) = ", labels.size(0))
        # labels.size(0) = 100
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print("Accuracy of the network on the 10000 test images: {} %".format(100 * correct / total))   

Accuracy of the network on the 10000 test images: 97.84 %


In [18]:
# Save the model checkpoint
torch.save(model.state_dict(), 'simple_nn.ckpt')